In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

/workspace/Recipes/rec_proj_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.94s/it]


GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [2]:
inputs = ["Как приготовить карбонару?"]
for query in inputs:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print(output)
    print()
    print("==============================")
    print()

/workspace/Recipes/rec_proj_venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Как приготовить карбонару?
Карбонара (Carbonara) — это итальянский рецепт, который традиционно готовится из яиц, пармезана, гуанчиале (или других видов копченого мяса), пасты спагетти и белого вина. Вот пошаговый рецепт приготовления:

Ингредиенты:
- 300 г спагетти
- 4 яйца
- 100 г пармезана
- 50 г гуанчиале или другого копченого мяса
- 1 столовая ложка белого вина
- соль по вкусу
- черный перец по вкусу

Приготовление:
1. **Подготовьте ингредиенты**: Налейте воду в кастрюлю и доведите до кипения. Откиньте лишнюю воду и добавьте немного оливкового масла для смазывания.
2. **Готовьте спагетти**: Влейте в кастрюлю спагетти и варите на среднем огне около 8-10 минут, пока они не станут аль денте. Затем откиньте их на тарелку и оставьте остывать.
3. **Приготовьте соус**: Разогрейте сковороду на среднем огне и добавьте гуанчиале. Обжарьте его до золотистой корочки, затем выньте мясо из сковороды и положите его на тарелку.
4. **Добавьте яйца и сыр**: В сковороде оставьте только масло, в котор